In [19]:
import pandas as pd
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from cnn_class import CNN  # Make sure to import your CNN class
import torch.nn.functional as F

In [7]:

def load_model(model_path):
    model = CNN(num_classes=16)  # Adjust num_classes if needed
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

def preprocess_image(image_array):
    image = Image.fromarray(image_array.reshape(28, 28).astype(np.uint8))
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    return transform(image).unsqueeze(0)

def predict(model, image_tensor):
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = output.max(1)
    return predicted.item()


In [11]:
import torch
import pandas as pd
import numpy as np
from cnn_class import CNN  # Make sure this import is correct

def load_model(model_path):
    model = CNN(num_classes=16)
    model.load_state_dict(torch.load(model_path))
    return model

def preprocess_image(image_array):
    # Reshape from (784,) to (1, 1, 28, 28)
    image = image_array.reshape(1, 1, 28, 28).astype(np.float32)
    return torch.from_numpy(image)

# Load the test data
test_data = pd.read_csv('test_data.csv')

# Load the model
model_path = 'best_model_cnn2.pth'  # Use the same file as in the working code
model = load_model(model_path)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define class mapping
class_mapping = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: '+', 11: '-', 12: '*', 13: '/', 14: '(', 15: ')'
}

# Process test samples
num_samples = 20
correct = 0
total = 0

with torch.no_grad():
    for i in range(num_samples):
        sample = test_data.iloc[i]
        ground_truth = sample.iloc[-1]
        image_array = sample.iloc[:-1].values
        image_tensor = preprocess_image(image_array).to(device)
        
        output = model(image_tensor)
        _, predicted = output.max(1)
        prediction = predicted.item()
        
        correct += (prediction == ground_truth)
        total += 1
        
        print(f"Sample {i+1}:")
        print(f"Ground Truth: {class_mapping[ground_truth]}")
        print(f"Prediction: {class_mapping[prediction]}")
        print(f"Correct: {prediction == ground_truth}")
        print("---")

accuracy = correct / total
print(f"Sample Accuracy: {accuracy:.2%}")

# Calculate overall accuracy
total_correct = 0
total_samples = len(test_data)

with torch.no_grad():
    for i in range(total_samples):
        sample = test_data.iloc[i]
        ground_truth = sample.iloc[-1]
        image_array = sample.iloc[:-1].values
        image_tensor = preprocess_image(image_array).to(device)
        
        output = model(image_tensor)
        _, predicted = output.max(1)
        prediction = predicted.item()
        
        total_correct += (prediction == ground_truth)

overall_accuracy = total_correct / total_samples
print(f"Overall Accuracy: {overall_accuracy:.2%}")

Sample 1:
Ground Truth: (
Prediction: (
Correct: True
---
Sample 2:
Ground Truth: 4
Prediction: 4
Correct: True
---
Sample 3:
Ground Truth: *
Prediction: *
Correct: True
---
Sample 4:
Ground Truth: 0
Prediction: 0
Correct: True
---
Sample 5:
Ground Truth: 9
Prediction: 9
Correct: True
---
Sample 6:
Ground Truth: -
Prediction: -
Correct: True
---
Sample 7:
Ground Truth: 5
Prediction: 5
Correct: True
---
Sample 8:
Ground Truth: 5
Prediction: 5
Correct: True
---
Sample 9:
Ground Truth: 3
Prediction: 3
Correct: True
---
Sample 10:
Ground Truth: -
Prediction: -
Correct: True
---
Sample 11:
Ground Truth: 8
Prediction: 8
Correct: True
---
Sample 12:
Ground Truth: (
Prediction: (
Correct: True
---
Sample 13:
Ground Truth: )
Prediction: )
Correct: True
---
Sample 14:
Ground Truth: 0
Prediction: 0
Correct: True
---
Sample 15:
Ground Truth: *
Prediction: *
Correct: True
---
Sample 16:
Ground Truth: 5
Prediction: 5
Correct: True
---
Sample 17:
Ground Truth: *
Prediction: *
Correct: True
---
Sample

testing individual images


In [ ]:
def predict_elements(image, bounding_boxes, model):
    results = []
    device = next(model.parameters()).device  # Get the device of the model
    
    for i, bbox in enumerate(bounding_boxes):
        element_tensor = preprocess_element(image, bbox).to(device)
        
        with torch.no_grad():
            output = model(element_tensor)
            probabilities = F.softmax(output, dim=1)
            predicted_class = probabilities.argmax(1).item()
            confidence = probabilities[0][predicted_class].item()
            
        results.append((predicted_class, confidence))
    return results

In [20]:
image_path = "./handwritten-digits-and-operators/CompleteImages/All data (Compressed)/6/6_1_0.png"
symbols = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '%', '*', '[', ']']


model_path = 'best_model_cnn2.pth'  # Use the same file as in the working code
model = load_model(model_path)
model.eval()


for symbol in symbols:

    img = Image.open(image_path)
    # Convert to grayscale
    img = img.convert("L")
    img = img.resize((28, 28))
    img_array = np.array(img)
    img_array = 255 - img_array
    img_tensor = torch.from_numpy(img_array).float().unsqueeze(0).unsqueeze(0)
    
    with torch.no_grad():
        output = model(img_tensor)
        probabilities = F.softmax(output, dim=1)
        predicted_class = probabilities.argmax(1).item()    
        print(predicted_class)




6
